In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd


In [4]:
#Load trained model,scaler pickle and one hot encoder
model= load_model('model.h5')

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder.pkl', 'rb') as file:
    one_hot_encoder = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [83]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'Germany',
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 1,
    'Balance': 0,
    'NumOfProducts': 1,
    'HasCrCard': 0,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [84]:
#One-hot encode categorical features
geo_encoded= one_hot_encoder.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df


c:\Users\rahul.bc.gupta\Desktop\KN ML\ANN project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [85]:
input_df = pd.DataFrame([input_data])

In [86]:
input_df['Gender']=label_encoder_gender.transform([input_data['Gender']])
input_df = pd.concat([input_df, geo_encoded_df], axis=1)
input_df = input_df.drop(columns=['Geography'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,35,1,0,1,0,1,50000,0.0,1.0,0.0


In [87]:
#Scaling data
input_df_scaled = scaler.transform(input_df)

In [88]:
input_df_scaled

array([[-0.53598516,  0.91324755, -0.37056859, -1.38944225, -1.21847056,
        -0.91668767, -1.54035103,  0.97481699, -0.87683221, -0.99850112,
         1.72572313, -0.57638802]])

In [89]:
#Predict churn
predictions = model.predict(input_df_scaled)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


array([[0.18286638]], dtype=float32)

In [90]:
# Check if the prediction is greater than 0.5 for churn
if predictions[0][0] > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
